<a href="https://colab.research.google.com/github/JisungJang-AI-BIO/LAIDD_Developability/blob/main/1_Boltz_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 소개 및 설정

**실습: Boltz-2를 활용한 항체-항원 구조 예측**

**목표**: 이 노트북에서는 Boltz-2를 사용하여 유방암 치료 항체인 트라스투주맙(Trastuzumab)과 그 표적 항원인 HER2 단백질의 결합 구조를 예측합니다. 아미노산 서열 정보만을 이용하여 복잡한 단백질 복합체의 3차원 구조를 생성하고 시각화하는 과정을 경험할 것입니다.

**참고**: 이 실습은 MIT와 Recursion에서 개발한 Boltz-2 모델의 기능을 기반으로 합니다. Boltz-2는 항체-항원 복합체와 같은 까다로운 대상의 구조 예측에서 뛰어난 성능을 보입니다.

In [ ]:
# 1단계: 호환되는 PyTorch 및 Boltz 패키지 설치
# Colab 환경의 기존 패키지와의 충돌을 피하고, boltz에 필요한 버전을 명시적으로 설치합니다.
# 먼저 torch, torchvision을 특정 버전으로 설치하여 `RuntimeError: operator torchvision::nms does not exist` 에러를 해결합니다.

print("호환되는 PyTorch 버전 설치 중... (약 1분 소요)")
!pip install torch==2.2.0 torchvision==0.17.0 torchaudio==2.2.0 --index-url https://download.pytorch.org/whl/cu121 -q

print("\nBoltz 패키지 설치 중...")
!pip install boltz -U -q

print("\n설치가 완료되었습니다.")

## 생물학적 문제 정의

**표적 항원 (Antigen)**: 인간 상피세포 성장인자 수용체 2(HER2)는 세포 성장을 조절하는 단백질입니다. 일부 유방암 및 위암에서 HER2가 과발현되어 통제되지 않는 세포 성장을 유발합니다.

**항체 (Antibody)**: 트라스투주맙(Trastuzumab, 상품명: 허셉틴)은 HER2에 특이적으로 결합하여 그 기능을 억제하는 인간화 단일클론항체입니다. 우리는 이 항체의 중쇄(Heavy Chain)와 경쇄(Light Chain) 서열을 사용할 것입니다.

**우리의 목표**: Boltz-2에게 트라스투주맙의 중쇄와 경쇄, 그리고 HER2 항원이 어떻게 상호작용하여 3차원 복합체 구조를 형성하는지 예측하도록 요청할 것입니다.

In [ ]:
# 2단계: 항체와 항원의 아미노산 서열을 정의합니다.

# 트라스투주맙(Trastuzumab) 항체의 가변 영역(variable region) 서열
# 출처: DrugBank, KEGG
heavy_chain_sequence = "EVQLVESGGGLVQPGGSLRLSCAASGFNIKDTYIHWVRQAPGKGLEWVARIYPTNGYTRYADSVKGRFTISADTSKNTAYLQMNSLRAEDTAVYYCSRWGGDGFYAMDYWGQGTLVTVSS"
light_chain_sequence = "DIQMTQSPSSLSASVGDRVTITCRASQDVNTAVAWYQQKPGKAPKLLIYSASFLYSGVPSRFSGSRSGTDFTLTISSLQPEDFATYYCQQHYTTPPTFGQGTKVEIK"

# HER2 단백질의 세포 외 도메인(extracellular domain) 서열
# 출처: UniProt P04626 (residues 23-652)
antigen_sequence = "TQVCTGTDMKLRLPASPETHLDMLRHLYQGCQVVQGNLELTYLPTNASLSFLQDIQEVQGYVLIAHNQVRQVPLQRLRIVRGTQLFEDNYALAVLDNGDPLNNTTPVTGASPGGLRELQLRSLTEILKGGVLIQRNPQLCYQDTILWKDIFHKNNQLALTLIDTNRSRACHPCSPMCKGSRCWGESSEDCQSLTRTVCAGGCARCKGPLPTDCCHEQCAAGCTGPKHSDCLACLHFNHSGICELHCPALVTYNTDTFESMPNPEGRYTFGASCVTACPYNYLSTDVGSCTLVCPLHNQEVTAEDGTQRCEKCSKPCARVCYGLGMEHLREVRAVTSANIQEFAGCKKIFGSLAFLPESFDGDPASNTAPLQPEQLQVFETLEEITGYLYISAWPDSLPDLSVFQNLQVIRGRILHNGAYSLTLQGLGISWLGLRSLRELGSGLALIHHNTHLCFVHTVPWDQLFRNPHQALLHTANRPEDECVGEGLACHQLCARGHCWGPGPTQCVNCSQFLRGQECVEECRVLQGLPREYVNARHCLPCHPECQPQNGSVTCFGPEADQCVACAHYKDPPFCVARCPSGVKPDLSYMPIWKFPDEEGACQPCPINCTHSCVDLDDKGCPAEQRASPLT"

# 출력 파일 이름 지정
output_name = "Trastuzumab_HER2_prediction"

print("입력이 정의되었습니다:")
print(f"항체 중쇄 (Heavy Chain): {len(heavy_chain_sequence)} 아미노산")
print(f"항체 경쇄 (Light Chain): {len(light_chain_sequence)} 아미노산")
print(f"HER2 항원 (Antigen): {len(antigen_sequence)} 아미노산")

In [ ]:
# 3단계: Boltz는 예측 설정을 위해 YAML 파일을 사용합니다.
# 이 파일을 프로그래밍 방식으로 생성해 보겠습니다.
import yaml

input_data = {
    'sequences': {
        'H': heavy_chain_sequence,
        'L': light_chain_sequence,
        'A': antigen_sequence
    },
    'name': output_name
}

yaml_path = f"{output_name}.yaml"
with open(yaml_path, 'w') as f:
    yaml.dump(input_data, f)

print(f"YAML 입력 파일이 생성되었습니다: {yaml_path}")
!cat {yaml_path} # 방금 생성한 파일의 내용을 표시합니다

In [ ]:
# 4단계: Boltz 예측을 실행합니다.
# 이 명령어는 Boltz에게 우리가 생성한 YAML 파일을 사용하도록 지시합니다.
# 참고: 이 단계는 GPU가 필요하며 몇 분 정도 소요될 수 있습니다. Colab 환경에서는 '런타임' > '런타임 유형 변경'에서 GPU를 활성화해야 합니다.
print("\nBoltz-2 예측을 시작합니다... 잠시만 기다려 주세요.")
!boltz predict {yaml_path}
print("예측이 완료되었습니다!")

## 분석: 3D 구조 시각화

**출력 해석**: Boltz-2는 이제 예측된 항체-항원 복합체의 3D 원자 좌표를 포함하는 PDB 파일을 생성했습니다.

**시각화**: py3Dmol 라이브러리를 사용하여 이 PDB 파일을 노트북에서 직접 로드하고 확인해 보겠습니다. 이를 통해 우리가 예측한 항체와 항원의 복잡한 상호작용을 눈으로 볼 수 있습니다.

In [ ]:
# 5단계: 예측된 구조를 시각화합니다.
# 먼저 py3Dmol을 설치합니다.
!pip install py3Dmol -q

import py3Dmol
import glob
import os

# 출력 디렉토리에서 PDB 파일을 찾습니다.
# glob.glob은 패턴과 일치하는 파일 경로 리스트를 반환하므로, 첫 번째 요소를 선택합니다.
try:
    # boltz는 'name'으로 지정된 폴더 안에 pdb를 저장합니다.
    pdb_file_path = os.path.join(output_name, 'unrelaxed_model_1.pdb')

    if not os.path.exists(pdb_file_path):
        raise FileNotFoundError

    with open(pdb_file_path, 'r') as f:
        pdb_data = f.read()

    view = py3Dmol.view(width=800, height=600)
    view.addModel(pdb_data, "pdb")

    # 각 체인을 다른 색상으로 시각화합니다.
    # 체인 ID는 YAML 파일에 정의한 'H', 'L', 'A'에 해당합니다.
    view.setStyle({'chain':'A'}, {'cartoon': {'color':'red'}})   # 항원(Antigen)은 빨간색
    view.setStyle({'chain':'H'}, {'cartoon': {'color':'blue'}})  # 중쇄(Heavy Chain)는 파란색
    view.setStyle({'chain':'L'}, {'cartoon': {'color':'green'}}) # 경쇄(Light Chain)는 녹색

    view.zoomTo()
    view.show()
except FileNotFoundError:
    print(f"오류: 예측된 PDB 파일({pdb_file_path})을 찾을 수 없습니다. 이전 예측 단계가 성공적으로 실행되었는지 확인하세요.")
except Exception as e:
    print(f"시각화 중 오류가 발생했습니다: {e}")

## 결론 및 다음 단계

**요약**: 축하합니다! 👏 여러분은 Boltz-2를 사용하여 종단 간 인실리코(in silico) 항체-항원 구조 예측 작업을 성공적으로 수행했습니다. 항체와 항원의 아미노산 서열 정보만을 제공하여, 그들의 3D 상호작용 복합체 구조를 예측하고 시각화했습니다.

**더 넓은 맥락**: 여러분이 몇 분 만에 완료한 이 작업 흐름은 전통적인 구조생물학 방법(예: X선 결정학)으로는 수개월 또는 수년이 걸릴 수 있는 작업입니다. 이러한 가속화는 Boltz-2와 같은 도구가 현대 항체 신약 개발 및 단백질 공학 연구를 혁신하는 이유입니다.

**추가 학습**:
- **공식 Boltz GitHub 저장소 탐색하기**: https://github.com/jwohlwend/boltz
- 다른 항체나 항원 서열로 **2단계**의 내용을 수정하여 예측이 어떻게 변하는지 직접 확인해 보세요!